In [1]:
import datetime
from ib_insync import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

Creating library 'pnl' that will keep track of strategy & account PnL


In [24]:
ac.delete_library('pnl')

In [17]:
ib = connect_to_IB(clientid=2)

Creating library 'pnl' that will keep track of strategy & account PnL


Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))

In [25]:
ac = initialize_db('db')
ac

Creating library 'pnl' that will keep track of strategy & account PnL


Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))

In [26]:
pnl_lib = ac.get_library('pnl')


In [27]:
total_equity =  sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")
total_equity

105213.53

In [35]:
def get_total_equity(ib_client):
    """Fetches the total equity value from the account summary."""
    total_equity = sum(float(entry.value) for entry in ib_client.accountSummary() if entry.tag == "EquityWithLoanValue")
    return total_equity

def save_pnl(ac, total_equity, account_id):
    """Saves the PnL (equity value) to the ArcticDB."""
    pnl_lib = ac.get_library('pnl')
     # Truncate the current timestamp to an hourly granularity
    current_time = datetime.datetime.now().replace(second=0, microsecond=0)
    
    pnl_data = {
        'total_equity': total_equity,
        'account_id': account_id
    }
    pnl_df = pd.DataFrame([pnl_data], index=[current_time])
    try:
        # Append the new data to the 'pnl' library, creating it if it doesn't exist
        if 'pnl' in pnl_lib.list_symbols():
            pnl_lib.append(account_id, pnl_df)
        else:
            pnl_lib.write(account_id, pnl_df)
        print(f"Equity value saved to 'pnl' library for account {account_id}")
    except Exception as e:
        print(f"Error saving equity value to 'pnl' library: {e}")


In [36]:
account_id = ib.managedAccounts()[0]  # Assuming you want the first managed account's ID
total_equity = get_total_equity(ib)
save_pnl(ac, total_equity, account_id)

Equity value saved to 'pnl' library for account U7706434


In [37]:
pnl_data = pnl_lib.read(account_id).data
pnl_data

,total_equity,account_id
2024-03-26 15:00:00,105212.87,U7706434
